## Importing the libs

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import json, requests
!pip install -q pygsheets
import pygsheets
import os
import time
import csv
from ds.utilities.io import ds_trino as ds_presto

from shapely import wkt

from IPython.core.display import HTML

from google.cloud import storage




You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


# Paths

In [2]:
path_cred = '/home/guilherme.augusto/Credencials/client_secret.json'

In [3]:
path_result = '/home/guilherme.augusto/Examples/'

# Example code for SUPERSET


First you set you SQL code in a string, then you execute the code to make the db select

In [4]:
query = """
SELECT DISTINCT
o.facility_name,
--o.facility_id
o.facility_latitude as latitude ,
o.facility_longitude as longitude
FROM hudi_ingest.analytics_views.customer_orders o
WHERE day_partition >= '2020-12-31'
and o.facility_licensee_type = 'CSS'
and o.facility_country_code in ('BR')
ORDER BY 1 ASC 
"""


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
df = ds_presto.fetch_data(sql_str=query, use_cache=False)
df.head()

,facility_name,latitude,longitude
0,BR - Sao Paulo - Acre,-23.55986213684082,-46.58323669433594
1,BR - Sao Paulo - Borba,-23.65675163269043,-46.70024871826172
2,BR - Sao Paulo - Clélia,-23.524967193603516,-46.700138092041016
3,BR - Sao Paulo - Dino,-23.52906036376953,-46.6451530456543
4,BR - Sao Paulo - Getulio Vargas,-23.694746017456055,-46.54399871826172


# READ AND WRITE FROM SHEETS

First you have to autenticate, then you can use this example

## GoogleSheets authentication

In [7]:

gc = pygsheets.authorize(service_file= path_cred)

# Write

to write is almosts the same, just follow the code bellow
remember to open by the key, set the correct sheet  and where the paste will start


In [8]:
# open the spreedsheet 
aux3 = gc.open_by_key('1bO0-ltYcctN2fqUV70yuyADV3vvTL-Kw_ihoIpharHA')

# Select in what worksheet will paste
auxs3 = aux3.worksheet('title','example 1')
# Clear worksheet
auxs3.clear(start='A1')
# select where and paste
auxs3.set_dataframe(df,start="A1",copy_head=True,empty_value='')

## READ

To Read follow this example, first you have to get the key of the spreadsheet and share the sheet with the service account email

In [9]:
exdf = gc.open_by_key('1bO0-ltYcctN2fqUV70yuyADV3vvTL-Kw_ihoIpharHA')
exdf2 = exdf.worksheet('title','example 1')
dataframe = exdf2.get_as_df(has_header=True,start="A1", end=None,empty_value='')

In [10]:
dataframe.head()

,facility_name,latitude,longitude
0,BR - Sao Paulo - Acre,-23.559862,-46.583237
1,BR - Sao Paulo - Borba,-23.656752,-46.700249
2,BR - Sao Paulo - Clélia,-23.524967,-46.700138
3,BR - Sao Paulo - Dino,-23.529060,-46.645153
4,BR - Sao Paulo - Getulio Vargas,-23.694746,-46.543999


# SEND FOR GOOGLE CLOUD

## saving in a csv

In [11]:
df_csv = df.to_csv(f'{path_result}df.csv')

## Send for google cloud

In [12]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=path_cred
gcs_s = storage.Client()
BUCKET ='css-brazil'
mypath = f'{path_result}df.csv'
bucket_folder = 'example_fergal/'
save_location= bucket_folder + 'df.csv'



### function to send 

In [13]:
gcs_s.get_bucket(BUCKET).blob(save_location).upload_from_filename(mypath,content_type='text/csv')

# READ FILES IN GOOGLE CLOUD 

In [14]:
df_gc = pd.read_csv('gs://css-brazil/example_fergal/df.csv',
                 storage_options={"token": '/home/guilherme.augusto/Credencials/client_secret.json'},index_col=0)




In [15]:
df_gc.head()

,facility_name,latitude,longitude
0,BR - Sao Paulo - Acre,-23.559862,-46.583237
1,BR - Sao Paulo - Borba,-23.656752,-46.700249
2,BR - Sao Paulo - Clélia,-23.524967,-46.700138
3,BR - Sao Paulo - Dino,-23.529060,-46.645153
4,BR - Sao Paulo - Getulio Vargas,-23.694746,-46.543999


# SEND FOR BIG QUERY 

In [16]:
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery

In [17]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/home/guilherme.augusto/Credencials/client_secret.json'
key_path = '/home/guilherme.augusto/Credencials/client_secret.json'
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
client = bigquery.Client()

In [18]:
# import the dataframe 

df_gc = pd.read_csv('gs://css-brazil/example_fergal/df.csv',
                 storage_options={"token": '/home/guilherme.augusto/Credencials/client_secret.json'},index_col=0)




In [19]:
df_gc.to_gbq(credentials = credentials,
              destination_table='css_brazil.fergal_ex',
              progress_bar = True,
              if_exists='replace'
              #if_exists='append'
                        )

1it [00:04,  4.66s/it]
